<a href="https://colab.research.google.com/github/ihasdapie/roadSignClassification/blob/master/Traffic_Sign_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# grab datasets
!wget -q https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip
!wget -q https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip
!wget -q https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip
#unpack data
!unzip -qq /content/GTSRB_Final_Test_GT.zip
!unzip -qq /content/GTSRB_Final_Test_Images.zip
!unzip -qq /content/GTSRB_Final_Training_Images.zip


In [ ]:
# grab processed dataset
!cp /content/drive/My\ Drive/GTSRB/Processed_GTSRB_Dataset/GTSRB_Processed.zip /content/Processed_GTSRB.zip
!unzip -qq Processed_GTSRB.zip

# grab test csv
!wget -q https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip
!unzip -qq /content/GTSRB_Final_Test_GT.zip


In [ ]:
# imports
import glob
import numpy as np
from skimage import color, exposure, transform, io
import keras
import pandas
import os
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD, adam
from keras import backend as K
import time
import cv2
import pandas as pd

In [ ]:
# Constants
NUM_CLASSES = 43
IMAGE_SIZE=48  

In [ ]:
def getLabel(img_path):
    return int(img_path.split('/')[-2])

In [ ]:
# import images & labels

BASE_TRAINING_DIR = "/content/P_GTSRB/Final_Training/Images/"

TRAINING_PATHS = glob.glob(os.path.join(BASE_TRAINING_DIR, '*/*.ppm'))

np.random.shuffle(TRAINING_PATHS)

TRAINING_LABELS = [getLabel(path) for path in TRAINING_PATHS]

In [ ]:
images = []
for i in TRAINING_PATHS:
    images.append(cv2.imread(i))

In [ ]:
X = np.array(images, dtype='float32')
Y = np.eye(NUM_CLASSES, dtype='uint8')[TRAINING_LABELS]

In [ ]:
K.set_image_data_format('channels_last')
# change to channels last and imgsize, imgsize, 3 later
def cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), activation='relu'))
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    
    return model


In [ ]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
def lr_schedule(epoch):
    return lr*(0.1**int(epoch/10))

batch_size=32
epochs=30


In [ ]:
from keras.optimizers import SGD

model2 = cnn_model()

lr = 0.0001

model2.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])



In [ ]:
# with adam
model2 = cnn_model()

lr = 0.0001

model2.compile(loss='categorical_crossentropy',
              optimizer=adam(lr=lr),
              metrics=['accuracy'])



In [ ]:
# train model
model2.fit(X, Y, batch_size=batch_size, epochs=epochs,validation_split=0.2,
        #   callbacks=[ModelCheckpoint('model.h5', save_best_only=True)])

          callbacks=[LearningRateScheduler(lr_schedule),
                     ModelCheckpoint('model2.h5', save_best_only=True)])


Train on 31367 samples, validate on 7842 samples
Epoch 1/30
31367/31367 [==============================] - 26s 839us/step - loss: 1.9588 - accuracy: 0.4627 - val_loss: 0.6708 - val_accuracy: 0.8101
Epoch 2/30
31367/31367 [==============================] - 26s 822us/step - loss: 0.7100 - accuracy: 0.7875 - val_loss: 0.4240 - val_accuracy: 0.8783
Epoch 3/30
31367/31367 [==============================] - 26s 824us/step - loss: 0.4772 - accuracy: 0.8567 - val_loss: 0.2987 - val_accuracy: 0.9102
Epoch 4/30
31367/31367 [==============================] - 26s 826us/step - loss: 0.3628 - accuracy: 0.8907 - val_loss: 0.2520 - val_accuracy: 0.9297
Epoch 5/30
31367/31367 [==============================] - 26s 833us/step - loss: 0.2957 - accuracy: 0.9104 - val_loss: 0.2158 - val_accuracy: 0.9383
Epoch 6/30
31367/31367 [==============================] - 26s 824us/step - loss: 0.2449 - accuracy: 0.9244 - val_loss: 0.1946 - val_accuracy: 0.9445
Epoch 7/30
31367/31367 [==============================] -

In [ ]:
# train model
model.fit(X, Y, batch_size=batch_size, epochs=epochs,validation_split=0.2,
        #   callbacks=[ModelCheckpoint('model.h5', save_best_only=True)])

          callbacks=[LearningRateScheduler(lr_schedule),
                     ModelCheckpoint('model.h5', save_best_only=True)])


Train on 31367 samples, validate on 7842 samples
Epoch 1/30
31367/31367 [==============================] - 23s 733us/step - loss: 0.4739 - accuracy: 0.8575 - val_loss: 0.3331 - val_accuracy: 0.9045
Epoch 2/30
31367/31367 [==============================] - 23s 730us/step - loss: 0.4597 - accuracy: 0.8614 - val_loss: 0.3270 - val_accuracy: 0.9059
Epoch 3/30
31367/31367 [==============================] - 23s 729us/step - loss: 0.4475 - accuracy: 0.8650 - val_loss: 0.3189 - val_accuracy: 0.9069
Epoch 4/30
31367/31367 [==============================] - 23s 732us/step - loss: 0.4514 - accuracy: 0.8610 - val_loss: 0.3124 - val_accuracy: 0.9132
Epoch 5/30
31367/31367 [==============================] - 23s 732us/step - loss: 0.4266 - accuracy: 0.8704 - val_loss: 0.3101 - val_accuracy: 0.9115
Epoch 6/30
31367/31367 [==============================] - 23s 735us/step - loss: 0.4197 - accuracy: 0.8727 - val_loss: 0.2997 - val_accuracy: 0.9143
Epoch 7/30
31367/31367 [==============================] -

In [ ]:
!cp /content/model.h5 /content/drive/My\ Drive/GTSRB/model1.h5 

In [ ]:
!cp /content/model2.h5 /content/drive/My\ Drive/GTSRB/model2.h5 

In [ ]:
!ls /mnt

In [ ]:
test_entries = pd.read_csv('/content/GT-final_test.csv', sep=';')
BASE_TEST_DIR = "/content/P_GTSRB/Final_Test/Images/"

X_test = []
y_test = []
i = 0

for file_name, class_id in zip(list(test_entries['Filename']), list(test_entries['ClassId'])):
    img_path = os.path.join(BASE_TEST_DIR, file_name)
    X_test.append(cv2.imread(img_path))
    y_test.append(class_id)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
predictions = model.predict_classes(X_test)
acc = np.sum(predictions== y_test)/np.size(predictions)
print(acc)

0.8569279493269992


In [ ]:
predictions = model2.predict_classes(X_test)
acc = np.sum(predictions== y_test)/np.size(predictions)
print(acc)

0.8912905779889153
